In [1]:
from statsmodels.tsa.stattools import innovations_algo
from statsmodels.tsa.arima_process import arma_acf, arma_acovf
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

from statsmodels.tools.validation import (
    array_like,
    bool_like,
    int_like,
)

In [ ]:
def levinson_durbin(s, nlags=10, isacov=False):
    s = array_like(s, "s")
    nlags = int_like(nlags, "nlags")
    isacov = bool_like(isacov, "isacov")
    order = nlags

    if isacov:
        sxx_m = s

    phi = np.zeros((order + 1, order + 1), "d")
    sig = np.zeros(order + 1)
    # initial points for the recursion
    phi[1, 1] = sxx_m[1] / sxx_m[0]
    sig[1] = sxx_m[0] - phi[1, 1] * sxx_m[1]
    for k in range(2, order + 1):
        phi[k, k] = (
            sxx_m[k] - np.dot(phi[1:k, k - 1], sxx_m[1:k][::-1])
        ) / sig[k - 1]
        for j in range(1, k):
            phi[j, k] = phi[j, k - 1] - phi[k, k] * phi[k - j, k - 1]
        sig[k] = sig[k - 1] * (1 - phi[k, k] ** 2)

    sigma_v = sig[-1]
    arcoefs = phi[1:, -1]
    pacf_ = np.diag(phi).copy()
    pacf_[0] = 1.0
    return arcoefs, sigma_v

In [8]:
#Ex: X_t - 0.4X_(t-1) + 0.7X_(t-2) = Z_t + 0.3Z_(t-1) - 0.4Z_(t-2)
#ar_coff = [1, 0.4, -0.7]
#ma_coff = [1, 0.3, -0.4]

alpha = 19
beta = 11
ar_coff = [1, 0]
ma_coff = [1, -0.19, 0.11]
sigma2 = (alpha/100)**2

lags = len(ma_coff)
acvf = np.ndarray.tolist(arma_acovf(ar_coff,ma_coff,lags,dtype=float,sigma2=sigma2))
acf = np.ndarray.tolist(arma_acf(ar_coff,ma_coff,lags+1))
print("ACVF \n",acvf)
print("ACF \n",acf)
ma_coff

ACVF 
 [0.03784002, -0.0076134900000000005, 0.003971]
ACF 
 [1.0, -0.2012020606754436, 0.10494180499904598, 0.0]


[1, -0.19, 0.11]

In [7]:
#durbin - levinson
print("acvf: ", acvf)
for i in range(1,6):
    if i < len(acvf):
        print(levinson_durbin(acvf[:i+1], nlags= len(acvf[:i+1]) - 1, isacov=True))
    else:
        acvf.append(0)
        print(levinson_durbin(acvf, nlags= len(acvf) - 1, isacov=True))

acvf:  [0.042207722899999994, -0.015289432999999998, 0.0011913]
(array([-0.36224255]), 0.03666923976468996)
(array([-0.40518684, -0.11855122]), 0.036153875897503344)
(array([-0.40954762, -0.13345559, -0.03678394]), 0.03610495760811314)
(array([-0.40995863, -0.13494676, -0.04136003, -0.01117353]), 0.036100449983276946)
(array([-0.40999625, -0.13508603, -0.04181445, -0.01255403, -0.0033674 ]), 0.03610004062575246)


In [9]:
#Innovation_algorithm
for i in range(1,5):
    if i < len(acvf):
        theta, v = innovations_algo(acvf[:i+1]) 
        #print("theta = ", theta[1:, :], "v: ", v[-1])
        #print()
    else:
        acvf.append(0)
        theta, v = innovations_algo(acvf)
        #print("theta = ", theta[1:, :], "v: ", v[-1])
        #print()
print("acvf: ", acvf)
print("Theta =\n", theta[1:, :])
print("v = ", v[1:])


acvf:  [0.03784002, -0.0076134900000000005, 0.003971, 0, 0]
Theta =
 [[-0.20120206  0.        ]
 [-0.18768549  0.1049418 ]
 [-0.19002136  0.10936932]
 [-0.18999443  0.10986515]]
v =  [0.03630817 0.03614431 0.03610061 0.03610059]
